In [1]:
from fastapi import FastAPI, Depends, status
from starlette.exceptions import HTTPException
from fastapi.middleware.cors import CORSMiddleware
from starlette.status import HTTP_422_UNPROCESSABLE_ENTITY

import asyncio

In [2]:
from typing import Optional

from pydantic import BaseModel

from typing import Any, Dict, AnyStr, List, Union

JSONObject = Dict[AnyStr, Any]
JSONArray = List[Any]
JSONStructure = Union[JSONArray, JSONObject]

In [3]:
from fastapi.security import APIKeyHeader
#access_token = APIKeyHeader(name="X-API-KEY")

app = FastAPI(title="Eason_Model_API_Mockup")


app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [4]:
import adapter_model

Torch Version: 1.8.1
transformers (Adapter) Version: 2.0.1
labels: ['B-art' 'B-eve' 'B-geo' 'B-gpe' 'B-nat' 'B-org' 'B-per' 'B-tim'
 'CountryCode' 'CryptoCurrencyCode' 'CurrencyCode' 'Event' 'Float' 'I-art'
 'I-eve' 'I-geo' 'I-gpe' 'I-nat' 'I-org' 'I-per' 'I-tim' 'Integer'
 'Location' 'Month' 'O' 'Object' 'Party' 'Race' 'SpecialTerm'
 'TemporalUnit' 'Time' 'Timezone' 'US_States']


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

In [5]:
from fastapi.security import APIKeyHeader
#access_token = APIKeyHeader(name="X-API-KEY")

app = FastAPI(title="Eason_Model_API_Mockup", docs_url=None, redoc_url=None, openapi_url = None)


app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [6]:
# Will customize OpenAPI Afterward to change the document's order
from fastapi.openapi.docs import get_swagger_ui_html
from fastapi.openapi.utils import get_openapi
@app.get("/docs", include_in_schema=False)
async def get_documentation():
    return get_swagger_ui_html(openapi_url="/openapi.json", title="docs")


@app.get("/openapi.json", include_in_schema=False)
async def openapi():
    return get_openapi(title = "FastAPI", version="0.1.0", routes=app.routes)


openAPI = get_openapi(title = "FastAPI", version="0.1.0", routes=app.routes)

import collections

paths = collections.OrderedDict()

In [7]:
example_text = "Dan Will be deemed to have completed its delivery obligations before 2021-7-5 if in Niall's opinion, the Jeep Car satisfies the Acceptance Criteria, and Niall notifies Dan in writing that it is accepting the Jeep Car."

In [8]:
@app.get("/model/label", tags = ["Status"], status_code=status.HTTP_200_OK)
def get_model_status():
    return {
        "message": "get success",
        "status": "re-training",
        "step": {
            "current": 30,
            "total": 50,
        },
        "train-data-amount": 2500,
        "affect-labels": ['Party', 'Race', 'SpecialTerm', 'TemporalUnit', 'Time', 'Timezone', 'US_States']
    }

In [9]:
@app.get("/model/template", tags = ["Status"], status_code=status.HTTP_200_OK)
def get_model_status():
    return {
        "message": "get success",
        "status": "re-training",
        "step": {
            "current": 30,
            "total": 50,
        },
        "train-data-amount": 2500,
        "affect-labels": ['Party', 'Race', 'SpecialTerm', 'TemporalUnit', 'Time', 'Timezone', 'US_States']
    }

In [10]:
class text_label_body(BaseModel):
    text: str = example_text

In [11]:
@app.post("/model/label", tags = ["Predict"], status_code=status.HTTP_200_OK)
def text_label(data: text_label_body):
    sen, pred, logits, logits_order = adapter_model.model_predict(adapter_model.model, data.text)
    out_Tokens = []
    for i, _ in enumerate(sen):
        predictions = []
        for j, _ in enumerate(logits_order):
            predictions.append({
                "type": logits_order[j],
                "confidence": logits[i][j],
            })
        predictions.sort(key = lambda x: x["confidence"], reverse=True)
        out_Tokens.append({
            "token": sen[i],
            "predictions": predictions,
        })
    return out_Tokens

In [12]:
@app.post("/model/template", tags = ["Predict"], status_code=status.HTTP_200_OK)
def recommand_template(data: text_label_body):
    return {
        "predict": [
            {
                "name": "Full Payment Upon Demand",
                "confidence": 0.9
            },
            {
                "name": "Acceptance of Delivery",
                "confidence": 0.8
            },
            {
                "name": "Copyright License",
                "confidence": 0.7
            },
            {
                "name": "Demand Forecast",
                "confidence": 0.6
            },
            {
                "name": "Eat Apples",
                "confidence": 0.5
            },
        ]
    }

In [13]:
class suggest_best_practice_body(BaseModel):
    text: str = example_text

In [14]:
@app.post("/model/suggestion", tags = ["Predict"], status_code=status.HTTP_200_OK)
def suggest_best_practice(data: suggest_best_practice_body):
    return {
        "predict": "Not Finish Yet."
    }

In [15]:
class update_data_body(BaseModel):
    text: JSONArray = [
        {
            "text": "Eason",
            "label": "Party"
        },
        {
            "text": "will",
            "label": "O"
        },
        {
            "text": "meet",
            "label": "O"
        },
        {
            "text": "Dan",
            "label": "Party"
        },
        {
            "text": "at",
            "label": "O"
        },
        {
            "text": "2021-08-04 18:00",
            "label": "TemporalUnit"
        },
        {
            "text": ".",
            "label": "O"
        },
    ]

In [16]:
openAPI["paths"].keys()

dict_keys([])

In [17]:
openAPI

{'openapi': '3.0.2',
 'info': {'title': 'FastAPI', 'version': '0.1.0'},
 'paths': {}}

In [18]:
@app.post("/data/label", tags = ["Optimize Data"], status_code=status.HTTP_200_OK)
def update_data(data: update_data_body):
    return "OK"

In [19]:
class update_template_data_body(BaseModel):
    text: str = example_text
    template: str = "IP Payment"

In [20]:
@app.post("/data/template", tags = ["Optimize Data"], status_code=status.HTTP_200_OK)
def update_template_data(data: update_template_data_body):
    return "OK"

In [21]:
class update_suggestion_data_body(BaseModel):
    text: str = "Not Finish Yet"

In [22]:
@app.post("/data/suggestion", tags = ["Optimize Data"], status_code=status.HTTP_200_OK)
def update_suggestion_data(data: update_suggestion_data_body):
    return "OK"

In [23]:
@app.post("/model/suggestion:retrain", tags = ["ReTrain"], status_code=status.HTTP_200_OK)
def retrain_suggestion_model():
    return {
        "message": "success, start retrain.",
        "train-data-amount": 2500,
    }

In [24]:
@app.post("/model/template:retrain", tags = ["ReTrain"], status_code=status.HTTP_200_OK)
def retrain_template_model():
    return {
        "message": "success, start retrain.",
        "train-data-amount": 2500,
    }

In [25]:
@app.post("/model/label:retrain", tags = ["ReTrain"], status_code=status.HTTP_200_OK)
def retrain_model():
    return {
        "message": "success, start retrain.",
        "train-data-amount": 2500,
        "affect-labels": ['Party', 'Race', 'SpecialTerm', 'TemporalUnit', 'Time', 'Timezone', 'US_States']
    }

In [ ]:
if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=13537)
    
    

INFO:     Started server process [14479]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:13537 (Press CTRL+C to quit)


INFO:     172.17.0.1:49952 - "POST /model/label HTTP/1.0" 200 OK
INFO:     172.17.0.1:49964 - "GET /docs HTTP/1.0" 200 OK
INFO:     172.17.0.1:49968 - "GET /openapi.json HTTP/1.0" 200 OK
